In [ ]:
#!pip install plotly
#!pip install ipywidgets
#!pip install bqplot ipyvuetify voila-vuetify

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')[['AAPL.Open','AAPL.Close']]

In [ ]:
def plot_corr(frequency, window):
    if not window or not frequency:
        return
    df_corr = (df
        .resample(frequency)
        .ffill()
        .rolling(window)
        .corr(pairwise=True)
        .unstack()['AAPL.Open']['AAPL.Close']
        )
    fig = px.line(x=df_corr.index, y=df_corr, title='Rolling correlation')
    fig.update_yaxes(range=(-1,1))
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

#plot_corr('D', 10)

In [ ]:
from ipywidgets import link,interactive, widgets, interact,Layout
config = {
    'layout': Layout(width='50%', height='80px'),
    'style': {'description_width': 'initial'}
}

# Interactive visualisation

<div class="alert alert-block alert-info">
    <b>Tip: use the widgets below to interact with the visualisation</b>
</div>


In [ ]:
frequency_selector = widgets.ToggleButtons(
    options=[('daily','D'), ('weekly',  'W'), ('monthly','30d')],
    value='D',
    description='Sample frequency:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    # icons=['check'] * 3
    **config,
)

window_selector = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='Rolling window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    **config,
)

_ = interact(plot_corr, frequency = frequency_selector,  window = window_selector)